In [1]:
import pandas as pd
from art_api import config, utils
import wandb
import os
from google.cloud import storage
from IPython.display import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential, layers, models
from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path
from wandb.keras import WandbCallback
from matplotlib import pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import Xception

%load_ext autoreload

In [2]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'baseline'

# Load dataset as artifact

In [21]:
# 1. Log a dataset version as an artifact

# Initialize a new W&B run to track this job
run = wandb.init(project="art_api", job_type="dataset-creation")
yourpaintings_sm_data = wandb.Artifact("yourpaintings_df",
                                       type="raw_data",
                                      description="YourPaintings dataset sm")
yourpaintings_sm_data.add_dir(config.PATH_YOURPAINTINGS_SM)
run.log_artifact(yourpaintings_sm_data)

wandb.finish()

wandb: Adding directory to artifact (./../raw_data/yourpaintings_sm)... Done. 6.5s


In [2]:
X, y = utils.load_data()

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

# Load baseline model

In [40]:
def load_baseline_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256,256,3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(10, activation='sigmoid'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [46]:
run = wandb.init(project="art_api", job_type="training")

In [49]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "baseline",
            type="model",
            description = "Simple baseline model")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = load_baseline_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "baseline")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [50]:
log_model()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


2022-09-17 00:20:22.771033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:20:22.771088: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 00:20:22.771239: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 00:20:22.771913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:20:22.772278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:20:22.772572: I tensorflow/stream_executor/cuda/cuda_gpu_exec

Epoch 1/50
181/182 [============================>.] - ETA: 0s - loss: 0.3647 - accuracy: 0.2490

INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_002021-1krtnfto/files/model-best/assets


...

182/182 [==============================] - 5s 27ms/step - loss: 0.2582 - accuracy: 0.4593 - val_loss: 0.2763 - val_accuracy: 0.4212
Epoch 18/50
182/182 [==============================] - 4s 22ms/step - loss: 0.2542 - accuracy: 0.4693 - val_loss: 0.2787 - val_accuracy: 0.4293
Epoch 19/50
182/182 [==============================] - 4s 22ms/step - loss: 0.2527 - accuracy: 0.4753 - val_loss: 0.2782 - val_accuracy: 0.4108
Epoch 20/50
182/182 [==============================] - 4s 22ms/step - loss: 0.2465 - accuracy: 0.4953 - val_loss: 0.2866 - val_accuracy: 0.3746
Epoch 21/50
182/182 [==============================] - 4s 23ms/step - loss: 0.2435 - accuracy: 0.5028 - val_loss: 0.2832 - val_accuracy: 0.4108
Epoch 22/50
182/182 [==============================] - 4s 22ms/step - loss: 0.2395 - accuracy: 0.5059 - val_loss: 0.2795 - val_accuracy: 0.4196
Epoch 23/50
182/182 [==============================] - 4s 22ms/step - loss: 0.2362 - accuracy: 0.5130 - val_loss: 0.2817 - val_accuracy: 0.4132
Epoc

INFO:tensorflow:Assets written to: ../models/baseline/assets


INFO:tensorflow:Assets written to: ../models/baseline/assets


39/39 [==============================] - 0s 10ms/step - loss: 0.2787 - accuracy: 0.4293


accuracy,▁▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
evaluate/accuracy,▁
loss,█▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
val_accuracy,▁▃▄▆▅▆▆▆▆▇▆▇▆▇▆▆██▇▅▇▇▇
val_loss,█▅▅▄▄▃▃▂▃▂▂▂▂▁▂▁▁▁▁▃▂▂▂
GFLOPs,0.49329
accuracy,0.513
best_epoch,16
best_val_loss,0.27628
epoch,22


# Sampling method 1: random undersampling

In [4]:
imgs, df = utils.init()

In [5]:
undersample_num = y.sum().min()
undersample_num

195

In [6]:
classes = ['aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'sheep', 'train']

In [7]:
y_sub = df.loc[:, ['aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train']]

In [8]:
y_sub

,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
8291,0,0,0,0,0,1,0,0,0,0
8292,0,0,1,0,0,0,0,0,0,0
8293,0,1,0,0,0,0,0,0,0,0
8294,0,1,0,0,0,0,0,0,0,0


In [9]:
df_aeroplane, df_bird, df_boat, df_chair, df_cow, df_diningtable, df_dog, df_horse, df_sheep, df_train = [y_sub[y_sub[cls] == 1] for cls in classes]

In [10]:
def undersample(n, cls_df):
    '''This function will undersample all other classes based on the minority class
    Args:
    n = number of samples to obtain
    X = class df, containing images from only 1 class
    y = target df
    Returns:
    '''
    cls_df = cls_df.sample(n)
    cls_X = X[cls_df.index]
    print(cls_df.index)

    return cls_df, cls_X

In [11]:
cls_aeroplane, aeroplane_X = undersample(195, df_aeroplane)
cls_bird, bird_X = undersample(195, df_bird)
cls_boat, boat_X = undersample(195, df_boat)
cls_chair, chair_X = undersample(195, df_chair)
cls_cow, cow_X = undersample(195, df_cow)
cls_diningtable, diningtable_X = undersample(195, df_diningtable)
cls_dog, dog_X = undersample(195, df_dog)
cls_horse, horse_X = undersample(195, df_horse)
cls_sheep, sheep_X = undersample(195, df_sheep)
cls_train, train_X = undersample(195, df_train)

Int64Index([ 662, 5224, 2221, 5584, 2971, 7630, 6419, 3851, 3148, 2964,
            ...
            3849, 3855, 3850, 2037, 4204, 2977, 5888,  548,  661, 5930],
           dtype='int64', length=195)
Int64Index([2726, 1687, 7094, 5368, 1683, 6376, 4994, 4850, 5950, 4770,
            ...
             264, 8049,  982, 7695, 4711, 4450,  993, 4771, 8056, 7244],
           dtype='int64', length=195)
Int64Index([ 483, 1084, 2659, 1288,  631, 4288, 3267, 4735, 7868, 1243,
            ...
             615, 1234,  741, 3619, 4796, 1278, 7323, 2813, 4882, 6807],
           dtype='int64', length=195)
Int64Index([4631,  678, 7793, 5970, 5098, 4513, 8114, 2254, 2894, 2900,
            ...
            2800, 2954, 6006, 5526, 2995, 2117, 5073, 3713,  810, 4592],
           dtype='int64', length=195)
Int64Index([ 825, 6622, 3984, 3231,  441, 1934,  207, 4004, 1907, 3234,
            ...
             815, 1847, 2417, 1879, 1851, 7817, 1857, 1834,  431, 1924],
           dtype='int64', length=195)
Int64

In [12]:
X_under = np.concatenate((aeroplane_X, bird_X, boat_X, chair_X, cow_X, diningtable_X, dog_X, horse_X, sheep_X, train_X), axis=0)

In [13]:
y_under = pd.concat([cls_aeroplane, cls_bird, cls_boat, cls_chair, cls_cow, cls_diningtable, cls_dog, cls_horse, cls_sheep, cls_train])

In [14]:
y_under

,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
662,1,0,0,0,0,0,0,0,0,0
5224,1,0,0,0,0,0,0,0,0,0
2221,1,0,0,0,0,0,0,0,0,0
5584,1,0,0,0,0,0,0,0,0,0
2971,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5151,0,0,0,0,0,0,0,0,0,1
7432,0,1,0,0,0,0,0,0,0,1
1312,0,0,0,0,0,0,0,0,0,1
2042,0,0,0,0,0,0,0,0,0,1


In [15]:
print(X_under.shape)
print(y_under.shape)

(1950, 256, 256, 3)
(1950, 10)


In [16]:
input_shape = X_train.shape[1:]

# Train baseline with balanced dataset

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [104]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "baseline-balanced",
            type="model",
            description = "Simple baseline model on balanced dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = load_baseline_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "baseline_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [105]:
log_model()

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/50


2022-09-17 00:51:13.538903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:51:13.538964: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 00:51:13.539088: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 00:51:13.539750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:51:13.540172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 00:51:13.540615: I tensorflow/stream_executor/cuda/cuda_gpu_exec

 6/43 [===>..........................] - ETA: 0s - loss: 0.6731 - accuracy: 0.0729WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0169s vs `on_train_batch_end` time: 0.0268s). Check your callbacks.


43/43 [==============================] - ETA: 0s - loss: 0.4861 - accuracy: 0.1128

...

43/43 [==============================] - 2s 44ms/step - loss: 0.3274 - accuracy: 0.3223 - val_loss: 0.3398 - val_accuracy: 0.2877
Epoch 14/50
43/43 [==============================] - 1s 23ms/step - loss: 0.3219 - accuracy: 0.3282 - val_loss: 0.3447 - val_accuracy: 0.2568
Epoch 15/50
43/43 [==============================] - 1s 24ms/step - loss: 0.3182 - accuracy: 0.3480 - val_loss: 0.3404 - val_accuracy: 0.2808
Epoch 16/50
43/43 [==============================] - 1s 24ms/step - loss: 0.3136 - accuracy: 0.3619 - val_loss: 0.3442 - val_accuracy: 0.2295
Epoch 16: early stopping


INFO:tensorflow:Assets written to: ../models/baseline_bal/assets


INFO:tensorflow:Assets written to: ../models/baseline_bal/assets


10/10 [==============================] - 0s 10ms/step - loss: 0.3414 - accuracy: 0.2911


accuracy,▁▁▁▂▃▅▅▆▆▆▇▇▇▇██
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
evaluate/accuracy,▁
loss,█▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
val_accuracy,▁▂▃▃▄▆▇▇▇██▇█▇█▆
val_loss,█▇▇▆▄▃▃▃▂▂▁▁▁▂▁▂
GFLOPs,0.49329
accuracy,0.3619
best_epoch,12
best_val_loss,0.3398
epoch,15


# Train on VGG16

In [125]:
def load_model():
        
    model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
        
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=(256,256,3))
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [126]:
def build_model():

    model = load_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [127]:
X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [128]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "vgg16-balanced",
            type="model",
            description = "VGG16 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "vgg16_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [129]:
log_model()

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
2022-09-17 01:04:00.937319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:04:00.937372: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 01:04:00.937498: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 01:04:00.938151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:04:00.938463: I tensorflow/stream_executor/cuda/cuda_gpu_

Epoch 1/50


2022-09-17 01:04:02.778778: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:04:02.778835: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


42/43 [============================>.] - ETA: 0s - loss: 0.4550 - accuracy: 0.3356

2022-09-17 01:04:07.778951: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.97GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:04:07.779000: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.97GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:04:07.819646: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:04:07.819684: W tensorflow/core/common_runtime/bfc_allocato

...

43/43 [==============================] - 10s 229ms/step - loss: 0.0834 - accuracy: 0.8593 - val_loss: 0.2246 - val_accuracy: 0.5856
Epoch 13/50
43/43 [==============================] - 6s 132ms/step - loss: 0.0755 - accuracy: 0.8564 - val_loss: 0.2252 - val_accuracy: 0.6027
Epoch 14/50
43/43 [==============================] - 5s 122ms/step - loss: 0.0686 - accuracy: 0.8630 - val_loss: 0.2261 - val_accuracy: 0.5753
Epoch 15/50
43/43 [==============================] - 7s 171ms/step - loss: 0.0624 - accuracy: 0.8586 - val_loss: 0.2256 - val_accuracy: 0.5719
Epoch 16/50
43/43 [==============================] - 5s 121ms/step - loss: 0.0570 - accuracy: 0.8630 - val_loss: 0.2268 - val_accuracy: 0.5959
Epoch 17/50
43/43 [==============================] - 5s 123ms/step - loss: 0.0521 - accuracy: 0.8608 - val_loss: 0.2275 - val_accuracy: 0.5788
Epoch 18/50
43/43 [==============================] - 5s 122ms/step - loss: 0.0478 - accuracy: 0.8623 - val_loss: 0.2292 - val_accuracy: 0.5856
Epoch 18: 

INFO:tensorflow:Assets written to: ../models/vgg16_bal/assets


INFO:tensorflow:Assets written to: ../models/vgg16_bal/assets


10/10 [==============================] - 1s 87ms/step - loss: 0.2252 - accuracy: 0.6027


accuracy,▁▄▆▆▇▇▇▇██████████
epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
evaluate/accuracy,▁
loss,█▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▆▅▆▇▇▇▇▇▇▇▇█▇▇█▇▇
val_loss,█▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,20.07383
accuracy,0.86227
best_epoch,11
best_val_loss,0.22464
epoch,17


# Train on VGG19

In [132]:
def load_vgg19_model():
        
    model = VGG19(weights="imagenet", include_top=False, input_shape=input_shape)
        
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=(256,256,3))
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [133]:
def build_vgg19_model():

    model = load_vgg19_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [135]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "vgg19-balanced",
            type="model",
            description = "VGG19 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_vgg19_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "vgg19_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [136]:
log_model()

80134624/80134624 [==============================] - 2s 0us/step


wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
2022-09-17 01:11:19.247039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:11:19.247088: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 01:11:19.247206: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 01:11:19.247867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:11:19.248197: I tensorflow/stream_executor/cuda/cuda_gpu_

Epoch 1/50
43/43 [==============================] - ETA: 0s - loss: 0.4475 - accuracy: 0.2864

INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_010959-3o1zk2j4/files/model-best/assets


INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_010959-3o1zk2j4/files/model-best/assets
wandb: Adding directory to artifact (/home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_010959-3o1zk2j4/files/model-best)... Done. 0.5s


...

43/43 [==============================] - 10s 227ms/step - loss: 0.0855 - accuracy: 0.8630 - val_loss: 0.2246 - val_accuracy: 0.5959
Epoch 13/50
43/43 [==============================] - 15s 348ms/step - loss: 0.0773 - accuracy: 0.8571 - val_loss: 0.2261 - val_accuracy: 0.6027
Epoch 14/50
43/43 [==============================] - 8s 170ms/step - loss: 0.0702 - accuracy: 0.8689 - val_loss: 0.2260 - val_accuracy: 0.6130
Epoch 15/50
43/43 [==============================] - 6s 143ms/step - loss: 0.0640 - accuracy: 0.8659 - val_loss: 0.2282 - val_accuracy: 0.6096
Epoch 16/50
43/43 [==============================] - 6s 142ms/step - loss: 0.0583 - accuracy: 0.8652 - val_loss: 0.2281 - val_accuracy: 0.5925
Epoch 17/50
43/43 [==============================] - 7s 166ms/step - loss: 0.0534 - accuracy: 0.8725 - val_loss: 0.2290 - val_accuracy: 0.6027
Epoch 18/50
43/43 [==============================] - 6s 138ms/step - loss: 0.0490 - accuracy: 0.8652 - val_loss: 0.2315 - val_accuracy: 0.5993
Epoch 19/

INFO:tensorflow:Assets written to: ../models/vgg19_bal/assets


INFO:tensorflow:Assets written to: ../models/vgg19_bal/assets


10/10 [==============================] - 1s 103ms/step - loss: 0.2260 - accuracy: 0.6130


accuracy,▁▄▆▆▇▇▇▇███████████
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
evaluate/accuracy,▁
loss,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▅▅▇▆▇▇▇▇██▇███▇█▇█
val_loss,█▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,25.5105
accuracy,0.8674
best_epoch,11
best_val_loss,0.22462
epoch,18


# Train on ResNet50

In [18]:
from tensorflow.keras.applications.resnet import preprocess_input

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [20]:
X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [149]:
def load_resnet_model():
    
    model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [150]:
def build_model():

    model = load_resnet_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [151]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "resnet50-balanced",
            type="model",
            description = "ResNet50 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "resnet50_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [152]:
log_model()

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
2022-09-17 01:20:03.731347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:20:03.731404: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 01:20:03.731475: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 01:20:03.732064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 01:20:03.732349: I tensorflow/stream_executor/cuda/cuda_gpu_

Epoch 1/50
 3/43 [=>............................] - ETA: 2s - loss: 0.6706 - accuracy: 0.1250

2022-09-17 01:20:21.674694: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:20:21.674757: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:20:21.690196: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-17 01:20:21.690262: W tensorflow/core/common_runtime/bfc_allocato

43/43 [==============================] - ETA: 0s - loss: 0.4341 - accuracy: 0.3267

INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_011958-89su9kk6/files/model-best/assets


...

43/43 [==============================] - 42s 992ms/step - loss: 0.0638 - accuracy: 0.8674 - val_loss: 0.2106 - val_accuracy: 0.6062
Epoch 18/50
43/43 [==============================] - 5s 103ms/step - loss: 0.0583 - accuracy: 0.8696 - val_loss: 0.2131 - val_accuracy: 0.6096
Epoch 19/50
43/43 [==============================] - 6s 148ms/step - loss: 0.0536 - accuracy: 0.8718 - val_loss: 0.2131 - val_accuracy: 0.6130
Epoch 20/50
43/43 [==============================] - 5s 107ms/step - loss: 0.0494 - accuracy: 0.8696 - val_loss: 0.2129 - val_accuracy: 0.6164
Epoch 20: early stopping


INFO:tensorflow:Assets written to: ../models/resnet50_bal/assets


INFO:tensorflow:Assets written to: ../models/resnet50_bal/assets


10/10 [==============================] - 1s 55ms/step - loss: 0.2110 - accuracy: 0.6164


accuracy,▁▄▅▆▆▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
evaluate/accuracy,▁
loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▆▅▆▇▇▇██▇▇▇████████
val_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,5.11001
accuracy,0.8696
best_epoch,16
best_val_loss,0.21055
epoch,19


# Train on ResNet152

In [31]:
def load_resnet152_model():
    
    model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [32]:
def build_model():

    model = load_resnet152_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [33]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "resnet152-balanced",
            type="model",
            description = "ResNet152 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "resnet152_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [34]:
log_model()

2022-09-17 02:55:12.828843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 02:55:12.893222: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 02:55:12.893585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 02:55:12.894430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/50


2022-09-17 02:55:42.365797: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-17 02:55:43.811475: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-17 02:55:43.812483: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-09-17 02:55:43.812504: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-09-17 02:55:43.812544: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

 2/43 [>.............................] - ETA: 5s - loss: 0.6833 - accuracy: 0.1250  

2022-09-17 02:55:44.563773: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

43/43 [==============================] - 74s 2s/step - loss: 0.0974 - accuracy: 0.8484 - val_loss: 0.2225 - val_accuracy: 0.5753
Epoch 12/50
43/43 [==============================] - 9s 200ms/step - loss: 0.0875 - accuracy: 0.8505 - val_loss: 0.2227 - val_accuracy: 0.5925
Epoch 12: early stopping


INFO:tensorflow:Assets written to: ../models/resnet152_bal/assets


INFO:tensorflow:Assets written to: ../models/resnet152_bal/assets


10/10 [==============================] - 1s 117ms/step - loss: 0.2353 - accuracy: 0.6096


accuracy,▁▅▆▇▇▇▇█████
epoch,▁▂▂▃▄▄▅▅▆▇▇█
evaluate/accuracy,▁
loss,█▅▄▄▃▃▂▂▂▁▁▁
val_accuracy,▁▄▆▆▇▇█▇█▇▇▇
val_loss,█▆▄▃▂▂▂▁▁▁▁▁
GFLOPs,14.81471
accuracy,0.85055
best_epoch,10
best_val_loss,0.22246
epoch,11


# Train on ResNet152V2

In [21]:
def load_resnet152v2_model():
    
    model = ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [22]:
def build_model():

    model = load_resnet152v2_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [23]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "resnet152v2-balanced",
            type="model",
            description = "ResNet152v2 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "resnet152v2_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [24]:
log_model()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find baseline.
wandb: Currently logged in as: erniesg (art_api). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2022-09-17 03:39:58.094231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 03:39:58.154764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 03:39:58.155123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA su

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/50


2022-09-17 03:40:55.071910: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-17 03:41:00.536504: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-17 03:41:00.537458: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-09-17 03:41:00.537479: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-09-17 03:41:00.538042: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

 2/43 [>.............................] - ETA: 5s - loss: 2.0697 - accuracy: 0.1250  

...

43/43 [==============================] - 68s 2s/step - loss: 0.2933 - accuracy: 0.4081 - val_loss: 0.3610 - val_accuracy: 0.3390
Epoch 9/50
43/43 [==============================] - 8s 183ms/step - loss: 0.2912 - accuracy: 0.4381 - val_loss: 0.4281 - val_accuracy: 0.2774
Epoch 10/50
43/43 [==============================] - 9s 220ms/step - loss: 0.2625 - accuracy: 0.4923 - val_loss: 0.4086 - val_accuracy: 0.2637
Epoch 11/50
43/43 [==============================] - 8s 184ms/step - loss: 0.2626 - accuracy: 0.4938 - val_loss: 0.4168 - val_accuracy: 0.3185
Epoch 12/50
43/43 [==============================] - 8s 176ms/step - loss: 0.2544 - accuracy: 0.5143 - val_loss: 0.3859 - val_accuracy: 0.2979
Epoch 13/50
43/43 [==============================] - 8s 193ms/step - loss: 0.2668 - accuracy: 0.5106 - val_loss: 0.3934 - val_accuracy: 0.3288
Epoch 13: early stopping


INFO:tensorflow:Assets written to: ../models/resnet152v2_bal/assets


INFO:tensorflow:Assets written to: ../models/resnet152v2_bal/assets


10/10 [==============================] - 1s 115ms/step - loss: 0.3610 - accuracy: 0.3390


accuracy,▁▃▄▄▅▆▆▆▇████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
evaluate/accuracy,▁
loss,█▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▃▇▆▅▇▆█▆▅▇▆█
val_loss,█▄▂▂▂▂▂▁▅▄▄▂▃
GFLOPs,14.31923
accuracy,0.51062
best_epoch,7
best_val_loss,0.36096
epoch,12


# Train on InceptionV3 

In [17]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [19]:
X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [20]:
def load_inceptionv3_model():
    
    model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [21]:
def build_model():

    model = load_inceptionv3_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [22]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "inceptionv3-balanced",
            type="model",
            description = "InceptionV3 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "inceptionv3_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [23]:
log_model()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erniesg (art_api). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2022-09-17 05:22:34.425525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 05:22:34.484312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 05:22:34.484574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA su

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/50


2022-09-17 05:22:44.840302: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-17 05:22:46.468281: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-17 05:22:46.469163: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-09-17 05:22:46.469182: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-09-17 05:22:46.469217: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

 3/43 [=>............................] - ETA: 1s - loss: 0.6823 - accuracy: 0.0312      

...

43/43 [==============================] - 26s 606ms/step - loss: 0.1734 - accuracy: 0.7473 - val_loss: 0.2271 - val_accuracy: 0.5822
Epoch 20: early stopping


INFO:tensorflow:Assets written to: ../models/inceptionv3_bal/assets


INFO:tensorflow:Assets written to: ../models/inceptionv3_bal/assets


10/10 [==============================] - 0s 38ms/step - loss: 0.2410 - accuracy: 0.5925


accuracy,▁▂▅▆▆▇▇▇▇▇▇▇▇███████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
evaluate/accuracy,▁
loss,█▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▄▆▆▇▇▇▇▇███████████
val_loss,█▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁
GFLOPs,3.8927
accuracy,0.74725
best_epoch,19
best_val_loss,0.22708
epoch,19


# Train on InceptionResNetV2 

In [24]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [26]:
X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [27]:
def load_inceptionresnetv2_model():
    
    model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [28]:
def build_model():

    model = load_inceptionresnetv2_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [29]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "inceptionresnetv2-balanced",
            type="model",
            description = "InceptionResNetV2 model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "inceptionresnetv2_bal")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [30]:
log_model()

219055592/219055592 [==============================] - 4s 0us/step


2022-09-17 05:35:58.342535: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1073479680 exceeds 10% of free system memory.
2022-09-17 05:35:58.837045: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1073479680 exceeds 10% of free system memory.
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
2022-09-17 05:36:00.126854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-17 05:36:00.127531: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-09-17 05:36:00.127628: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-09-17 0

Epoch 1/50
43/43 [==============================] - ETA: 0s - loss: 0.5237 - accuracy: 0.2755

INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_053548-3e1zn6pq/files/model-best/assets


INFO:tensorflow:Assets written to: /home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_053548-3e1zn6pq/files/model-best/assets
wandb: Adding directory to artifact (/home/erniesg/code/erniesg/art_api/notebooks/wandb/run-20220917_053548-3e1zn6pq/files/model-best)... Done. 1.0s


...

43/43 [==============================] - 62s 1s/step - loss: 0.1688 - accuracy: 0.7172 - val_loss: 0.2222 - val_accuracy: 0.5993
Epoch 22: early stopping


INFO:tensorflow:Assets written to: ../models/inceptionresnetv2_bal/assets


INFO:tensorflow:Assets written to: ../models/inceptionresnetv2_bal/assets


10/10 [==============================] - 1s 104ms/step - loss: 0.2318 - accuracy: 0.6062


accuracy,▁▃▃▅▆▆▆▆▇▇▇▇▇▇▇▇██████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
evaluate/accuracy,▁
loss,█▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▂▁▅▆▆▇▇███▇███████████
val_loss,█▆▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
GFLOPs,8.84396
accuracy,0.71722
best_epoch,21
best_val_loss,0.22224
epoch,21


# Train on Xception 

In [38]:
from tensorflow.keras.applications.xception import preprocess_input

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [40]:
X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [41]:
def load_xception_model():
    
    model = Xception(weights='imagenet', include_top=False, input_shape=input_shape, classes=10)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    
    return model

def set_nontrainable_layers(model):
    
    # Set the first layers to be untrainable
    model.trainable = False
        
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    rescale_layer = layers.Rescaling(1./255, input_shape=input_shape)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(10, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        rescale_layer,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [42]:
def build_model():

    model = load_xception_model()
    model = add_last_layers(model)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [43]:
def log_model():
    #run = wandb.init(project=“project="art_api", job_type="training")
    
    with wandb.init(project="art_api") as run:
        
        baseline_model = wandb.Artifact(
            name = "xception-balanced",
            type="model",
            description = "Xception model on balanced, preprocessed dataset")
    
        es = EarlyStopping(monitor = 'val_accuracy', 
                       mode = 'max', 
                       patience = 5, 
                       verbose = 1, 
                       restore_best_weights = True)
        
        # Define WandbCallback for experiment tracking
        wandb_callback = WandbCallback(monitor='val_loss',
                                       log_weights=True,
                                       log_evaluation=True,
                                       validation_steps=5)

        callbacks = [es, wandb_callback]

        model = build_model()

        model.fit(X_train, y_train,  validation_data=(X_test, y_test),
              callbacks=[callbacks], verbose=1, epochs=50)

        filename = os.path.join(Path("../models"), "xception")
        model.save(filename)
        
        loss, acc = model.evaluate(X_test, y_test)
        wandb.log({'evaluate/accuracy': acc})

In [44]:
log_model()

Problem at: /tmp/ipykernel_14510/2487830688.py 4 log_model


Traceback (most recent call last):
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 1076, in init
    run = wi.init()
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 669, in init
    backend.interface.publish_header()
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 94, in publish_header
    self._publish_header(header)
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/wandb/sdk/interface/interface_shared.py", line 264, in _publish_header
    self._publish(rec)
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/wandb/sdk/interface/interface_sock.py", line 51, in _publish
    self._sock_client.send_record_publish(record)
  File "/home/erniesg/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/w

Exception: problem

In [72]:
def sample_together(n, X, y):
    X = X.sample(n)
    return X, y.iloc[X.index]
df_test, y_test = sample_together(195, df_bird, y)

In [76]:
df_test

,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
6166,0,1,0,0,0,0,0,0,0,0
3144,0,1,0,0,0,0,0,0,0,0
348,0,1,0,0,0,0,0,0,0,0
7841,0,1,0,0,0,0,0,0,0,0
3869,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1386,0,1,0,0,0,0,0,0,0,0
932,0,1,0,0,0,0,0,0,0,0
5279,0,1,0,0,0,0,0,0,0,0
2751,0,1,0,0,0,0,0,0,0,0


In [77]:
y_test

,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
17,0,1,0,0,0,0,1,0,0,0
18,0,1,0,0,0,0,0,0,0,0
40,0,1,0,0,0,0,0,0,0,0
41,0,1,0,0,0,0,0,0,0,0
42,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8282,0,1,0,0,0,0,0,0,0,0
8283,0,1,1,0,0,0,0,0,0,0
8293,0,1,0,0,0,0,0,0,0,0
8294,0,1,0,0,0,0,0,0,0,0


In [59]:
sample_bird = df_bird.sample(undersample_num)
y_bird = y.iloc[sample_bird.index]

In [61]:
y.index

RangeIndex(start=0, stop=8296, step=1)

In [ ]:
df = df.sample(1000)
target = target[df.index,]

In [58]:
y_test

,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
55,0,0,1,0,0,0,0,0,0,0
359,0,0,0,0,0,0,0,0,1,0
680,0,0,0,1,0,1,0,0,0,0
452,0,0,0,0,0,0,1,0,0,0
129,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
485,0,0,1,0,0,0,0,0,0,0
534,0,0,0,0,0,0,0,0,1,0
701,0,0,0,0,1,0,0,0,0,0
77,0,0,1,0,0,0,0,0,0,0


In [ ]:
def sample_together(195, X, y):
    rows = random.sample(np.arange(0,len(X.index)).tolist(),n)
    return X.iloc[rows,], y.iloc[rows,]
  
df_sample, target_sample = sample_together(195, df, target)

In [ ]:
def sample_together(n, args):
    rows = random.sample(np.arange(0,len(X.index)).tolist(),n)
    return tuple(arg.iloc[rows,] for arg in args)

df_sample, target_sample, target2_sample = sample_together(2, [df, target, target2])

In [46]:
sample_bird = df_bird.sample(undersample_num)
y_bird = y.iloc[sample_bird.index]

In [93]:
#from google.cloud import storage as gcs
import cloudstorage as gcs
import google-cloud-storage

gcs_file = gcs.open(filename)
contents = gcs_file.read()
gcs_file.close()
self.response.write(contents)

SyntaxError: invalid syntax (3223910753.py, line 3)

In [102]:
cls2

,filename,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
17,DUR_DBM_770-001.jpg,0,1,0,0,0,0,1,0,0,0
18,DUR_DBM_769-001.jpg,0,1,0,0,0,0,0,0,0,0
40,VA_PC_2007BP2389-001.jpg,0,1,0,0,0,0,0,0,0,0
41,LAN_TURT_PCF21-001.jpg,0,1,0,0,0,0,0,0,0,0
42,CDN_LBCN_426-001.jpg,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8282,NY_YAG_YORAG_189-001.jpg,0,1,0,0,0,0,0,0,0,0
8283,NY_NRM_1976_9315-001.jpg,0,1,1,0,0,0,0,0,0,0
8293,NY_YAG_YORAG_1030-001.jpg,0,1,0,0,0,0,0,0,0,0
8294,GL_GM_A_1989_23_e-001.jpg,0,1,0,0,0,0,0,0,0,0


In [104]:
cls3

,filename,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
21,SYO_CG_CP_TR_156-001.jpg,0,0,1,0,0,0,0,0,0,0
38,NG_NG_NG818-001.jpg,0,0,1,0,0,0,0,0,0,0
51,KT_MMB_06_014-001.jpg,0,0,1,0,0,0,0,0,0,0
52,NMM_NMMG_BHC1081-001.jpg,0,0,1,0,0,0,0,0,0,0
53,NWM_BSI_PCF2-001.jpg,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8279,NY_YAG_YORAG_1263-001.jpg,0,0,1,0,0,0,0,0,0,0
8280,NY_YAG_YORAG_615-001.jpg,0,0,1,0,0,0,0,0,0,0
8283,NY_NRM_1976_9315-001.jpg,0,1,1,0,0,0,0,0,0,0
8285,NOT_NCMG_1946_258-001.jpg,0,0,1,0,0,0,0,0,0,0


In [82]:
# Create individual dataframes per class

df_aeroplane = y_sub[y_sub['aeroplane'] == 1]
df_bird = y_sub[y_sub['bird'] == 1]
df_boat = y_sub[y_sub['boat'] == 1]
df_chair = y_sub[y_sub['chair'] == 1]
df_cow = y_sub[y_sub['cow'] == 1]
df_diningtable = y_sub[y_sub['diningtable'] == 1]
df_dog = y_sub[y_sub['dog'] == 1]
df_horse = y_sub[y_sub['horse'] == 1]
df_sheep = y_sub[y_sub['sheep'] == 1]
df_train = y_sub[y_sub['train'] == 1]

x in lst for [loop]

In [86]:
df_dog.iloc[0]

filename       NY_YAG_YORAG_326-001.jpg
aeroplane                             0
bird                                  0
boat                                  0
chair                                 1
cow                                   0
diningtable                           1
dog                                   1
horse                                 0
sheep                                 0
train                                 0
Name: 15, dtype: object

In [87]:
sample_train = df_train.sample(195)

,filename,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
4289,NY_NRM_1978_1319-001.jpg,0,0,0,0,0,0,0,0,0,1
6083,NY_NRM_1975_8504-001.jpg,0,0,0,0,0,0,0,0,0,1
3034,NY_NRM_1979_7953-001.jpg,0,0,0,0,0,0,0,0,0,1
4726,NY_NRM_1984_666-001.jpg,0,0,0,0,0,0,0,0,0,1
6842,DUR_DCC_772-001.jpg,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6949,NML_MUL_RD00169-001.jpg,0,0,0,0,0,0,0,0,0,1
7014,NY_NRM_1975_8515-001.jpg,0,0,0,0,0,0,0,0,0,1
3039,NY_NRM_1975_8507-001.jpg,0,0,0,0,0,0,0,0,0,1
6956,NY_NRM_1975_8492-001.jpg,0,0,0,0,0,0,0,0,0,1


In [76]:
cls = ['aeroplane', 'bird']
classes = ['aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train']

In [80]:
cls

,filename,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
1,GMIII_MOSI_A1978_72_3-001.jpg,0,0,0,0,0,0,0,0,0,1
2,NY_NRM_1979_7964-001.jpg,0,0,0,0,0,0,0,0,0,1
3,CHE_CRHC_PCF40-001.jpg,0,0,0,0,0,0,0,0,0,1
6,NY_NRM_1986_9418-001.jpg,0,0,0,0,0,0,0,0,0,1
7,NY_NRM_2004_7349-001.jpg,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
8109,NY_NRM_1997_8476-001.jpg,0,0,0,0,0,0,0,0,0,1
8248,NY_NRM_1978_1526-001.jpg,0,0,0,0,0,0,0,0,0,1
8277,NY_NRM_1986_9410-001.jpg,0,0,0,0,0,0,0,0,0,1
8278,NY_NRM_1986_9413-001.jpg,0,0,0,0,0,0,0,0,0,1


In [77]:
# df1, df2 = [x for _, x in df.groupby(df['Sales'] == 1)]
# df.loc[lambda df: df['shield'] == 8]

for x in cls:
    print(x)
    a = y_sub[y_sub[x] == 1]

a

aeroplane
bird


,filename,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
17,DUR_DBM_770-001.jpg,0,1,0,0,0,0,1,0,0,0
18,DUR_DBM_769-001.jpg,0,1,0,0,0,0,0,0,0,0
40,VA_PC_2007BP2389-001.jpg,0,1,0,0,0,0,0,0,0,0
41,LAN_TURT_PCF21-001.jpg,0,1,0,0,0,0,0,0,0,0
42,CDN_LBCN_426-001.jpg,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8282,NY_YAG_YORAG_189-001.jpg,0,1,0,0,0,0,0,0,0,0
8283,NY_NRM_1976_9315-001.jpg,0,1,1,0,0,0,0,0,0,0
8293,NY_YAG_YORAG_1030-001.jpg,0,1,0,0,0,0,0,0,0,0
8294,GL_GM_A_1989_23_e-001.jpg,0,1,0,0,0,0,0,0,0,0


In [ ]:
df1, df2 = x for _, x in y_sub.loc(y_sub[x] == 1)

In [ ]:
# df1, df2 = [x for _, x in df.groupby(df['Sales'] == 1)]
# df.loc[lambda df: df['shield'] == 8]

In [67]:
for i in classes:
    print(i)
    y_cls = y_sub.loc[:, ['filename', 'aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train']]

y_cls

aeroplane


KeyError: '["[\'aeroplane\', \'bird\', \'boat\', \'chair\', \'cow\', \'diningtable\', \'dog\', \'horse\', \'sheep\', \'train\']"] not in index'

In [ ]:
df.loc[lambda df: df['shield'] == 8]

In [43]:
df.columns

Index(['index', 'Image URL', 'Web page URL', 'Subset', 'Labels', 'filename',
       'labels', 'aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train'],
      dtype='object')

In [11]:
print(X.shape)
print(y.shape)
y.head()

(8296,)
(8296, 10)


,aeroplane,bird,boat,chair,cow,diningtable,dog,horse,sheep,train
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0


In [ ]:
imgs = []
df = pd.read_csv(f"gs://{config.BUCKET_NAME}/{config.BUCKET_TRAIN_DATA_FILE}")

In [ ]:
for index, row in df.iterrows():
    img_file = str(row["filename"])
    image = Image.open(os.path.join(f"gs://{config.BUCKET_NAME}/{config.BUCKET_TRAIN_DATA_PATH}", img_file))
    image = image.resize((256, 256), Image.ANTIALIAS)
    #image.save(os.path.join(config.PATH_YOURPAINTINGS_SM, img_file))
    imgs.append(np.array(image))
return imgs, df

In [ ]:
def blur_offensive_images(data):
    file_data = data

    file_name = file_data["name"]
    bucket_name = file_data["bucket"]

    blob = storage_client.bucket(bucket_name).get_blob(file_name)
    blob_uri = f"gs://{bucket_name}/{file_name}"
    blob_source = vision.Image(source=vision.ImageSource(image_uri=blob_uri))

In [12]:
%cd ../raw_data/yourpaintings_sm
%ls

[Errno 2] No such file or directory: '../raw_data/yourpaintings_sm'
/Users/enjiaochen/code/erniesg/art_api/raw_data/yourpaintings_sm
ABD_AAG_002200-001.jpg
ABD_AAG_002751-001.jpg
ABD_AAG_003645-001.jpg
ABD_AAG_003739-001.jpg
ABD_AAG_003952-001.jpg
ABD_AAG_005011-001.jpg
ABD_AAG_010590-001.jpg
ABD_AAG_013539-001.jpg
ABD_AAG_2776-001.jpg
ABD_AAG_2812-001.jpg
ABD_AAG_2879-001.jpg
ABD_AAG_ABDAG002249-001.jpg
ABD_AAG_ABDAG003660-001.jpg
ABD_AAG_ABDAG010584-001.jpg
ABD_AAG_ABDAG013824-001.jpg
ABD_AAG_AG002197-001.jpg
ABD_AAG_AG002221-001.jpg
ABD_AAG_AG002252-001.jpg
ABD_AAG_AG002303-001.jpg
ABD_AAG_AG002568-001.jpg
ABD_AAG_AG002683.jpg
ABD_AAG_AG002749-001.jpg
ABD_AAG_AG003159-001.jpg
ABD_AAG_AG003737a-001.jpg
ABD_AAG_AG004478-001.jpg
ABD_AAG_AG006583-001.jpg
ABD_AAG_AG009637-001.jpg
ABD_AAG_AG011574-001.jpg
ABD_AAG_ag000003-001.jpg
ABD_AAG_ag000687-001.jpg
ABD_AAG_ag002220-001.jpg
ABD_AAG_ag002253-001.jpg
ABD_AAG_ag002290-001.jpg
ABD_AAG_ag002357-001.jpg
ABD_AAG_ag002423-001.jpg
ABD_AAG_ag0

In [13]:
imgs, df = utils.resize_rescale()

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/yourpaintings/df_yourpaintings.csv'

In [3]:
X, y = utils.load_data()

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/yourpaintings_sm/NID_QUB_QUB_264-001.jpg'

In [11]:
!cd ../raw_data/yourpaintings

In [16]:
df.drop(columns=['Image URL', 'Web page URL', 'Subset', 'Labels', 'filename',
       'labels']).sum()

aeroplane       195
bird            781
boat           2050
chair          1147
cow             603
diningtable    1161
dog            1108
horse          1448
sheep           721
train           324
dtype: int64

In [15]:
mlsmote.get_tail_label(df)

['Image URL', 'Web page URL', 'filename']

In [12]:
X_, y_ = mlsmote.create_dataset()

In [13]:
X_.shape
y_.shape

(1000, 5)

In [14]:
X_.shape

(1000, 10)

In [24]:
X_

,0,1,2,3,4,5,6,7,8,9
0,-1.700263,0.602477,-2.478509,0.433484,1.109780,-0.246891,0.525389,-1.902413,1.991422,-1.848972
1,2.267874,-0.350847,2.187252,-0.769722,0.601022,-0.234049,-1.844444,2.092082,2.299990,-2.585919
2,-1.207028,0.230167,-2.035403,-0.401339,0.595955,2.066251,-2.311038,-1.804673,2.197500,-2.106087
3,-1.326205,-1.417870,2.085587,-0.453313,0.457689,0.868153,0.784653,2.348514,2.546447,-2.829462
4,-0.781795,1.495530,-2.194913,-0.264658,0.906237,-0.633937,-0.907535,-2.706511,1.255595,-1.120307
...,...,...,...,...,...,...,...,...,...,...
995,-2.287050,-0.298746,1.586469,-0.464742,1.103363,1.050799,0.960210,-2.157864,2.600376,-2.887021
996,-0.027245,1.842409,1.793464,0.013021,0.980886,-0.037242,-0.309943,-2.104098,-2.168505,2.055803
997,0.833553,-0.130508,-1.954786,-0.074340,-0.410230,0.837448,-1.627014,-1.333642,2.935449,-2.876682
998,-1.088050,-0.213819,-2.207270,-1.106803,-1.096291,0.722919,0.833781,-2.433188,1.347741,-1.211900


In [15]:
y_

,class_0,class_1,class_2,class_3,class_4
0,0,0,0,0,1
1,0,0,1,0,0
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,0,0,1
...,...,...,...,...,...
995,1,0,0,0,0
996,0,1,0,0,0
997,0,0,0,0,1
998,0,0,0,0,1


In [16]:
mlsmote.get_tail_label(y_)

['class_1', 'class_3']

In [23]:
X_sub, y_sub = mlsmote.get_minority_instace(X_, y_)
X_res, y_res = mlsmote.MLSMOTE(X_sub, y_sub, 100)

In [17]:
mlsmote.get_tail_label(y)

['aeroplane', 'cow', 'train']

In [6]:
print(type(X))
print(type(y))
X = pd.DataFrame(X)

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
X.head()

,0
0,"[[[255, 145, 8], [255, 147, 10], [255, 149, 12..."
1,"[[[174, 166, 145], [172, 164, 145], [170, 161,..."
2,"[[[107, 174, 165], [107, 174, 165], [108, 174,..."
3,"[[[87, 94, 113], [87, 94, 113], [89, 95, 117],..."
4,"[[[151, 170, 166], [151, 170, 166], [151, 170,..."


In [9]:
from art_api import mlsmote

X_sub, y_sub = mlsmote.get_minority_instace(X, y)
#X_res, y_res = mlsmote.MLSMOTE(X_sub, y_sub, 100)

In [10]:
X_sub

,0
0,"[[[255, 145, 8], [255, 147, 10], [255, 149, 12..."
1,"[[[174, 166, 145], [172, 164, 145], [170, 161,..."
2,"[[[107, 174, 165], [107, 174, 165], [108, 174,..."
3,"[[[87, 94, 113], [87, 94, 113], [89, 95, 117],..."
4,"[[[171, 102, 61], [187, 120, 78], [182, 115, 7..."
...,...
1114,"[[[136, 136, 102], [137, 137, 103], [132, 132,..."
1115,"[[[236, 234, 219], [236, 234, 221], [236, 234,..."
1116,"[[[242, 253, 247], [242, 253, 247], [242, 253,..."
1117,"[[[37, 29, 6], [40, 32, 11], [51, 43, 24], [51..."


In [16]:
X_sub[0]

0       [[[255, 145, 8], [255, 147, 10], [255, 149, 12...
1       [[[174, 166, 145], [172, 164, 145], [170, 161,...
2       [[[107, 174, 165], [107, 174, 165], [108, 174,...
3       [[[87, 94, 113], [87, 94, 113], [89, 95, 117],...
4       [[[171, 102, 61], [187, 120, 78], [182, 115, 7...
                              ...                        
1114    [[[136, 136, 102], [137, 137, 103], [132, 132,...
1115    [[[236, 234, 219], [236, 234, 221], [236, 234,...
1116    [[[242, 253, 247], [242, 253, 247], [242, 253,...
1117    [[[37, 29, 6], [40, 32, 11], [51, 43, 24], [51...
1118    [[[103, 116, 124], [102, 115, 123], [100, 113,...
Name: 0, Length: 1119, dtype: object

In [17]:
X_sub[0].shape

(1119,)

In [32]:
y_sub.sum()

aeroplane      195
bird            31
boat            35
chair            5
cow            603
diningtable      9
dog             52
horse           62
sheep           86
train          324
dtype: int64

In [35]:
X_res, y_res = mlsmote.MLSMOTE(X_sub, y_sub, 100)

ValueError: setting an array element with a sequence.